# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-13e03078b5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("World Wide Web") 

Consider the following exploratory information need:

> Investigate the origins of the World Wide Web and related academic activities and scientists.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q466`| World Wide Web      | node |





Also consider

```
wd:Q466 ?p ?obj .
```

is the BGP to retrieve all **properties of World Wide Web**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Find the inventors of World Wide Web (return IRI and name).

2. Identify the BGP for hypertext system.

3. Find all the hypertext systems born before the 1980 (return the IRI and name of the system and the inception date).

4. Identify the BGP for computer scientist

5. Find how many computer scientists there are for each continent (consider their citizenship). Return the IRI and name of the continent and the number of scientists for each continent. 

6. Find all the computer scientists who thaught at the University of Cambridge (return IRI and name)

7. Find all the computer scientists who wrote at least 5 books (please consider only the instances of book, exclude "literary work" or other type of work related to books). Return the IRI and name of the computer scientist and the number of books. 

## Task 1
Find the inventors of World Wide Web (return IRI and name).

In [2]:
# query example
# I find inventor wdt:P61
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   wd:Q466 ?p ?o.
   # get the label
   ?p sc:name ?pname.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1430'), ('pname', 'OpenPlaques subject ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('pname', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pname', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2004'), ('pname', 'NALT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('pname', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pname', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND I

Final query for this task

In [3]:
# write your final query
#i find name of inventors
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q466 wdt:P61 ?inventor.
   # get the label
   ?inventor sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Tim Berners-Lee')]
[('name', 'Robert Cailliau')]
2


## Task 2
Identify the BGP for hypertext system.

In [15]:
# write your queries


queryString = """
SELECT DISTINCT ?hypertext ?hypertext_name
WHERE {
 
   # bind something
   wd:Q466 ?p ?hypertext.
   # get the label
   ?hypertext sc:name ?hypertext_name.
   FILTER REGEX(?hypertext_name, 'hypertext')

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('hypertext', 'http://www.wikidata.org/entity/Q65966993'), ('hypertext_name', 'hypertext system')]
1


Final query for this task

In [4]:
# write your final query
# hypertext is subject and we need to find object and predicate


queryString = """
SELECT DISTINCT ?o  ?oname ?p ?pname
WHERE {
 
   # bind something
   wd:Q65966993 ?p ?o.
   # get the label
   ?o sc:name ?oname.
   ?p sc:name ?pname.

}
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q131093'), ('oname', 'content management system'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('o', 'http://www.wikidata.org/entity/Q17155032'), ('oname', 'software category'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
2


In [17]:
# write your final query


queryString = """
SELECT DISTINCT  ?s ?sname ?p ?pname 
WHERE {
 
   # bind something
    ?s ?p wd:Q65966993.
   # get the label
   ?s sc:name ?sname.
   ?p sc:name ?pname.

}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6686945'), ('sname', 'wiki software'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q7246937'), ('sname', 'Problem-Oriented Medical Information System'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q466'), ('sname', 'World Wide Web'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q370979'), ('sname', 'Amigaguide'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q2385520'), ('sname', 'ENQUIRE'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q4994212'), ('sname', 'Hypertext Editing System'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('s', 'http://www.wikidata.org/

## Task 3
Find all the hypertext systems born before the 1980 (return the IRI and name of the system and the inception date).

In [82]:
# write your final query
# I want to know relation between world wide web and hypertext system how they are connected

queryString = """
SELECT DISTINCT ?p   ?pname 
WHERE {
 
   # bind something
   wd:Q466 ?p wd:Q65966993.
   # get the label
   ?p sc:name ?pname.

}

LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
1


In [18]:
# query example
# I want to find bgp world wide web in order to determine predicate for born and its value 
# here I find P571 for born  
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   wd:Q466 ?p ?o.
   # get the label
   ?p sc:name ?pname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1430'), ('pname', 'OpenPlaques subject ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('pname', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pname', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2004'), ('pname', 'NALT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('pname', 'FAST ID')]
10


In [113]:
# write your final query
# I find all instance of hyper text which are created before 1980

queryString = """
SELECT DISTINCT  ?name_of_instance (YEAR(?when) as ?date) 
WHERE {
 
    # bind something
    ?instance_of_hypertext wdt:P31 wd:Q65966993;
                          wdt:P571  ?when.
    # get the label
    ?instance_of_hypertext sc:name ?name_of_instance.
 
     FILTER(YEAR(?when) < 1980)
}
order by desc (?when)
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('name_of_instance', 'Aspen Movie Map'), ('date', '1978')]
[('name_of_instance', 'Electronic Document System'), ('date', '1978')]
[('name_of_instance', 'ZOG'), ('date', '1977')]
[('name_of_instance', 'File Retrieval and Editing System'), ('date', '1968')]
[('name_of_instance', 'Hypertext Editing System'), ('date', '1967')]
[('name_of_instance', 'oNLine System'), ('date', '1962')]
[('name_of_instance', 'Project Xanadu'), ('date', '1960')]
[('name_of_instance', 'Memex'), ('date', '1939')]
8


Final query for this task

## Task 4
Identify the BGP for computer scientist

In [23]:
# write your queries
# i find computer scientisit wd:Q82594

queryString = """
SELECT DISTINCT ?job ?job_name
WHERE {
 
   # bind something
   ?s wdt:P106 ?job;
      wdt:P31  wd:Q5.
   # get the label
   ?s sc:name ?sname.
   ?job sc:name ?job_name.
   FILTER REGEX(?job_name,'computer scientist')
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q82594'), ('job_name', 'computer scientist')]
1


Final query for this task

In [24]:
# write your queries
#I find bgp of computer scientist


queryString = """
SELECT DISTINCT ?p   ?pname ?o ?oname
WHERE {
 
   # bind something
   wd:Q82594 ?p  ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.

}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('pname', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pname', 'Library of Congress authority ID')]
10


## Task 5
Find how many computer scientists there are for each continent (consider their citizenship). Return the IRI and name of the continent and the number of scientists for each continent.

Final query for this task

In [27]:
# show the nationality and person names 
# we should find the person with job is P106 and his/her nationality P27


queryString = """
SELECT DISTINCT ?person ?pname ?nationality ?nname
WHERE {
      # bind something
      ?person wdt:P106  wd:Q82594;
              wdt:P27 ?nationality.
      # get the label
      ?person sc:name ?pname.
      ?nationality sc:name ?nname.
   
 
}

LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q8190500'), ('pname', 'Seinosuke Toda'), ('nationality', 'http://www.wikidata.org/entity/Q17'), ('nname', 'Japan')]
[('person', 'http://www.wikidata.org/entity/Q603999'), ('pname', 'Kevin Poulsen'), ('nationality', 'http://www.wikidata.org/entity/Q30'), ('nname', 'United States of America')]
[('person', 'http://www.wikidata.org/entity/Q11109023'), ('pname', 'Michael Barr'), ('nationality', 'http://www.wikidata.org/entity/Q16'), ('nname', 'Canada')]
[('person', 'http://www.wikidata.org/entity/Q20731777'), ('pname', 'Valeria de Paiva'), ('nationality', 'http://www.wikidata.org/entity/Q155'), ('nname', 'Brazil')]
[('person', 'http://www.wikidata.org/entity/Q20731777'), ('pname', 'Valeria de Paiva'), ('nationality', 'http://www.wikidata.org/entity/Q145'), ('nname', 'United Kingdom')]
[('person', 'http://www.wikidata.org/entity/Q24901829'), ('pname', 'Ernesto Damiani'), ('nationality', 'http://www.wikidata.org/entity/Q38'), ('nname', 'Ital

In [32]:
# from the last query I find that the USA is Q30 so we need to find relation between USA and its continent
#here we find continent is P30
queryString = """
SELECT DISTINCT ?p ?pname ?cname
WHERE {
    # bind something
     wd:Q30 ?p ?continent.
    # get the label
    ?continent sc:name ?cname.
    ?p sc:name ?pname.
    Filter regex(?pname,'continent','i')

}


LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('cname', 'North America')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('cname', 'Insular Oceania')]
2


In [61]:


queryString = """
SELECT DISTINCT  ?continent   ?cname (count(?person) as ?howmany)
WHERE {
    # bind something
    ?person wdt:P106  wd:Q82594;
            wdt:P27 ?nationality.
    ?nationality wdt:P30 ?continent.
    # get the label
    ?continent sc:name ?cname.

}
#group by (?cname)

"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('cname', 'Eurasia'), ('howmany', '23')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe'), ('howmany', '3487')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('cname', 'Insular Oceania'), ('howmany', '2482')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('cname', 'Asia'), ('howmany', '953')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('cname', 'Africa'), ('howmany', '76')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('cname', 'South America'), ('howmany', '92')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('cname', 'North America'), ('howmany', '2785')]
7


In [62]:


queryString = """
SELECT ?continent  ?continent_name (count(?country) as ?howmany)
WHERE {
?country wdt:P30 ?continent.
?continent sc:name ?continent_name
filter(?nationality=?country)
    {SELECT  ?nationality
    WHERE {
        # bind something
        ?person wdt:P106  wd:Q82594;
                wdt:P27 ?nationality.

    }}}
#group by (?continent)

"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continent_name', 'Eurasia'), ('howmany', '23')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continent_name', 'Europe'), ('howmany', '3487')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continent_name', 'Insular Oceania'), ('howmany', '2482')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continent_name', 'Asia'), ('howmany', '953')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continent_name', 'Africa'), ('howmany', '76')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continent_name', 'South America'), ('howmany', '92')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continent_name', 'North America'), ('howmany', '2785')]
7


## Task 6
Find all the computer scientists who thaught at the University of Cambridge (return IRI and name)

In [69]:
# write your queries
# we find  university cambridge is Q35794 and each person relation to university of cambridge  
queryString = """
SELECT DISTINCT  ?p  ?pname ?o ?oname
WHERE {
 
   # bind something
   ?person wdt:P106 wd:Q82594;
           wdt:P31  wd:Q5;
           ?p ?o.
    
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   filter regex(?oname,"University of Cambridge")
  
}

limit 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q35794'), ('oname', 'University of Cambridge')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer'), ('o', 'http://www.wikidata.org/entity/Q35794'), ('oname', 'University of Cambridge')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q41795465'), ('oname', 'honorary doctor of the University of Cambridge')]
3


Final query for this task

In [72]:
# wd:Q35794 university of cambridge
# wdt:P108 employer
queryString = """
SELECT DISTINCT ?s  ?sname 
WHERE {
 
    # bind something
    ?s wdt:P106 wd:Q82594;
       wdt:P108 wd:Q35794.
    # get the label
    ?s sc:name ?sname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q92944'), ('sname', 'David Wheeler')]
[('s', 'http://www.wikidata.org/entity/Q7251'), ('sname', 'Alan Turing')]
[('s', 'http://www.wikidata.org/entity/Q365578'), ('sname', 'Nathan Myhrvold')]
[('s', 'http://www.wikidata.org/entity/Q446862'), ('sname', 'David J. C. MacKay')]
[('s', 'http://www.wikidata.org/entity/Q92431724'), ('sname', 'Andrew Pitts')]
[('s', 'http://www.wikidata.org/entity/Q46633'), ('sname', 'Charles Babbage')]
[('s', 'http://www.wikidata.org/entity/Q7259'), ('sname', 'Ada Lovelace')]
[('s', 'http://www.wikidata.org/entity/Q62857'), ('sname', 'Maurice Wilkes')]
[('s', 'http://www.wikidata.org/entity/Q7176624'), ('sname', 'Peter Robinson')]
[('s', 'http://www.wikidata.org/entity/Q451770'), ('sname', 'Douglas Hartree')]
[('s', 'http://www.wikidata.org/entity/Q6135125'), ('sname', 'James H. Davenport')]
[('s', 'http://www.wikidata.org/entity/Q4707402'), ('sname', 'Alan Mycroft')]
[('s', 'http://www.wikidata.org/entity/Q10348

## Task 7
Find all the computer scientists who wrote at least 5 books (please consider only the instances of book, exclude "literary work" or other type of work related to books). Return the IRI and name of the computer scientist and the number of books.

In [74]:
# write your queries
# i find author is wdt:P50 from bgp of computer scientists
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE {
 
   # bind something
   ?person wdt:P106 wd:Q82594;
            ?p ?o.
   # get the label
   ?p sc:name ?pname.
  
}
order by ?pname
limit 10

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3314'), ('pname', '365chess player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2179'), ('pname', 'ACM Classification Code (2012)')]
[('p', 'http://www.wikidata.org/prop/direct/P864'), ('pname', 'ACM Digital Library author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3332'), ('pname', 'ACM Digital Library citation ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3901'), ('pname', 'ADAGP artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4114'), ('pname', 'ADK member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5463'), ('pname', 'AE member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4432'), ('pname', 'AKL Online artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9226'), ('pname', 'ARAE ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9795'), ('pname', 'ARPI author ID')]
10


In [87]:
# write your queries

queryString = """
SELECT DISTINCT ?instance_of_book_name ?person ?person_name  (count(?book) as ?booknumber) 
WHERE {
 
  # bind something
  ?person wdt:P106 wd:Q82594.
  ?book   wdt:P50 ?person;
          wdt:P31 ?instance_of_book.
  # get the label
  ?person sc:name ?person_name.
  ?book   sc:name ?book_name.
  ?instance_of_book  sc:name ?instance_of_book_name.
  filter regex (?instance_of_book_name ,".book")
  #filter (day(?booknumber)<"5"^^xsd:int)

}
order by desc(?booknumber)
limit 15

"""

print("Results")
x=run_query(queryString)

Results
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q20492519'), ('person_name', 'Olatz Arbelaitz'), ('booknumber', '12')]
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q16517758'), ('person_name', 'Iñaki Alegria'), ('booknumber', '11')]
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q50177430'), ('person_name', 'Kepa Sarasola Gabiola'), ('booknumber', '7')]
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q12253843'), ('person_name', 'Arantza Diaz de Ilarraza Sanchez'), ('booknumber', '3')]
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q46633'), ('person_name', 'Charles Babbage'), ('booknumber', '2')]
[('instance_of_book_name', 'scientific book'), ('person', 'http://www.wikidata.org/entity/Q105467898'), ('person_name', 'Aitzol Astigarraga'), ('booknumber', '2')]
[('insta

Final query for this task

In [ ]:
# write your final query